In [1]:
import pandas as pd
import pymongo

In [2]:
countrypop_under15 = "resources/countries_by_population_0to14.csv"
countrypop_from15to64 = "resources/countries_by_population_15to64.csv"
countrypop_over65 = "resources/countries_by_population_over65.csv"

covid19data = "resources/covid_19_data.csv"

In [3]:
under15_pd = pd.read_csv(countrypop_under15, header = 4)
from15to64_pd = pd.read_csv(countrypop_from15to64, header = 4)
over65_pd = pd.read_csv(countrypop_over65, header = 4)
covid19_pd = pd.read_csv(covid19data)

In [4]:
under15_df = pd.DataFrame(under15_pd)
from15to64_df = pd.DataFrame(from15to64_pd)
over65_df = pd.DataFrame(over65_pd)
covid19_df = pd.DataFrame(covid19_pd)

In [5]:
covid19_df.head()

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,1,01/22/2020,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
1,2,01/22/2020,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0
2,3,01/22/2020,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0
3,4,01/22/2020,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
4,5,01/22/2020,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0


In [6]:
# clean covid19 data to remove provincial level of analysis and dates

In [7]:
covid19_reduced = covid19_pd[["Country/Region", "Confirmed", "Deaths", "Recovered"]]
covid19_reduced.head()

,Country/Region,Confirmed,Deaths,Recovered
0,Mainland China,1.0,0.0,0.0
1,Mainland China,14.0,0.0,0.0
2,Mainland China,6.0,0.0,0.0
3,Mainland China,1.0,0.0,0.0
4,Mainland China,0.0,0.0,0.0


In [8]:
# group by country/region and sum confirmed, deaths, and recovered

In [9]:
covid19_bycountry = covid19_reduced.groupby(["Country/Region"], as_index=False).sum()
covid19_bycountry.head()

,Country/Region,Confirmed,Deaths,Recovered
0,Azerbaijan,1.0,0.0,0.0
1,"('St. Martin',)",2.0,0.0,0.0
2,Afghanistan,74.0,0.0,0.0
3,Albania,160.0,5.0,0.0
4,Algeria,286.0,10.0,40.0


In [10]:
# arrange new dataframe by most confirmed covid19 cases by country and drop non-countries
covid19_bycountry_desc = covid19_bycountry.sort_values(["Confirmed"], ascending = False)
covid19_drop = covid19_bycountry_desc.drop([110])

covid19_drop.reset_index(inplace=True)
covid19_reindexed = covid19_drop.drop(["index"], axis=1)

In [11]:
# make new dataframe of top 10 countries only
covid19_topten = pd.DataFrame(covid19_reindexed.iloc[:10])

# add a "country code" column for later df merge
country_code = ["CHN","ITA","KOR","IRN","ESP","DEU","FRA","USA","JPN","CHE"]
covid19_topten["Country Code"] = country_code

# format index and column order
covid19_topten.index = covid19_topten.index + 1
covid19_topten.index.name = "Index"
covid19_topten_final = covid19_topten[["Country/Region","Country Code","Confirmed","Deaths","Recovered"]]
covid19_topten_final

,Country/Region,Country Code,Confirmed,Deaths,Recovered
Index,,,,,
1,Mainland China,CHN,2877598.0,93346.0,1220639.0
2,Italy,ITA,142850.0,8582.0,12239.0
3,South Korea,KOR,112920.0,808.0,3359.0
4,Iran,IRN,102312.0,4190.0,28609.0
5,Spain,ESP,29317.0,827.0,1759.0
6,Germany,DEU,24990.0,37.0,531.0
7,France,FRA,24632.0,486.0,282.0
8,US,USA,15572.0,380.0,222.0
9,Japan,JPN,9992.0,197.0,1506.0


clean and merge population datasets

In [12]:
# remove unwanted columns & rename
under15_df_latest = under15_df[["Country Name", "Country Code", "2018"]]
under15_df_renamed = under15_df_latest.rename(columns = {"2018": "Pop % 14 and under (2018)"})

from15to64_df_latest = from15to64_df[["Country Name", "Country Code", "2018"]]
from15to64_df_renamed = from15to64_df_latest.rename(columns = {"2018": "Pop % 15-64 (2018)"})

over65_df_latest = over65_df[["Country Name", "Country Code", "2018"]]
over65_df_renamed = over65_df_latest.rename(columns = {"2018": "Pop % 65 and over (2018)"})

In [13]:
# Select only top 10 covid19 countries and drop rest
under15_top10covid = under15_df_renamed.loc[(under15_df_renamed["Country Name"] == "China") | (
    under15_df_renamed["Country Name"] == "Italy") | (under15_df_renamed["Country Name"] == "Korea, Rep.") | (
    under15_df_renamed["Country Name"] == "Iran, Islamic Rep.") | (under15_df_renamed["Country Name"] == "Spain") | (
    under15_df_renamed["Country Name"] == "Germany") | (under15_df_renamed["Country Name"] == "France") | (
    under15_df_renamed["Country Name"] == "United States") | (under15_df_renamed["Country Name"] == "Japan") | (
    under15_df_renamed["Country Name"] == "Switzerland"), :]

from15to64_top10covid19 = from15to64_df_renamed.loc[(from15to64_df_renamed["Country Name"] == "China") | (
    from15to64_df_renamed["Country Name"] == "Italy") | (from15to64_df_renamed["Country Name"] == "Korea, Rep.") | (
    from15to64_df_renamed["Country Name"] == "Iran, Islamic Rep.") | (from15to64_df_renamed["Country Name"] == "Spain") | (
    from15to64_df_renamed["Country Name"] == "Germany") | (from15to64_df_renamed["Country Name"] == "France") | (
    from15to64_df_renamed["Country Name"] == "United States") | (from15to64_df_renamed["Country Name"] == "Japan") | (
    from15to64_df_renamed["Country Name"] == "Switzerland"), :]

over65_top10covid19 = over65_df_renamed.loc[(over65_df_renamed["Country Name"] == "China") | (
    over65_df_renamed["Country Name"] == "Italy") | (over65_df_renamed["Country Name"] == "Korea, Rep.") | (
    over65_df_renamed["Country Name"] == "Iran, Islamic Rep.") | (over65_df_renamed["Country Name"] == "Spain") | (
    over65_df_renamed["Country Name"] == "Germany") | (over65_df_renamed["Country Name"] == "France") | (
    over65_df_renamed["Country Name"] == "United States") | (over65_df_renamed["Country Name"] == "Japan") | (
    over65_df_renamed["Country Name"] == "Switzerland"), :]

In [14]:
# merge all ages
from14to64 = pd.merge(under15_top10covid, from15to64_top10covid19[["Country Code", "Pop % 15-64 (2018)"]], on="Country Code")
all_ages = pd.merge(from14to64, over65_top10covid19[["Country Code", "Pop % 65 and over (2018)"]], on="Country Code")

# format index
all_ages.index=all_ages.index+1
all_ages.index.name="Index"

create final table

In [15]:
# merge population and covid19 dataframes
merged_df = pd.merge(covid19_topten_final, all_ages, on="Country Code")

# final formatting
merged_df_drop=merged_df.drop(["Country Name"], axis=1)
final_df=merged_df_drop.rename(columns={"Country/Region": "Country Name"})
final_df.at[3,"Country Name"]="Iran"
final_df.at[7,"Country Name"]="United States"
final_df.index=final_df.index+1
final_df.index.name="Index"
final_df

,Country Name,Country Code,Confirmed,Deaths,Recovered,Pop % 14 and under (2018),Pop % 15-64 (2018),Pop % 65 and over (2018)
Index,,,,,,,,
1,Mainland China,CHN,2877598.0,93346.0,1220639.0,17.877003,71.202114,10.920884
2,Italy,ITA,142850.0,8582.0,12239.0,13.329116,63.919204,22.751680
3,South Korea,KOR,112920.0,808.0,3359.0,12.973324,72.608121,14.418556
4,Iran,IRN,102312.0,4190.0,28609.0,24.476559,69.338868,6.184574
5,Spain,ESP,29317.0,827.0,1759.0,14.667001,65.954491,19.378508
6,Germany,DEU,24990.0,37.0,531.0,13.621030,64.917008,21.461962
7,France,FRA,24632.0,486.0,282.0,17.956464,62.008911,20.034625
8,United States,USA,15572.0,380.0,222.0,18.709040,65.483306,15.807654
9,Japan,JPN,9992.0,197.0,1506.0,12.696849,59.726781,27.576370


export final table to database

In [16]:
# connect to mongodb
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
# translate df to dictionary
data_dict = final_df.to_dict("records")

In [17]:
db = client.covid19DB
covid19_countries_top10affected = db.covid19_countries_top10affected

In [18]:
covid19_countries_top10affected.insert_many(data_dict)